In [ ]:
import pyvips
import pandas as pd
import numpy as np
import pandas as pd
import plotnine as p9
import scanpy as sc
import glob
import json
from plotnine_prism import *
from tqdm import tqdm
import yaml
import squidpy as sq
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
import sys 
import anndata as ad
sys.path.append("../")
from src.preprocess_utils.preprocess_image import crop_tile
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_recall_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [ ]:
files = glob.glob("../TCGA_KIRC/data/kidney_tls_anno/*.json")
len(files)

In [ ]:
data = []
for path in tqdm(files):
    with open(path, 'r') as file:
        anno = json.load(file)
    if len(anno) > 0:

        for row in anno:
            if 'classification' in row['properties']:
                label = row['properties']['classification']['name']
                #if "TLS" not in label:
                #    continue
                for i in row['geometry']['coordinates']:
                    if row['geometry']['type'] == "Polygon":
                        coord = pd.DataFrame(i, columns=["x", "y"])
                        coord['label'] = label#"TLS" if "TLS" in label else label
                        coord["file_id"] = path.split("/")[-1].replace(".json", "")
                        data.append(coord)
                    else:
                        for j in i:
                        
                            coord = pd.DataFrame(np.array(i[0]), columns=["x", "y"])
                            coord['label'] = label
                            coord["file_id"] = path.split("/")[-1].replace(".json", "")
                            data.append(coord)
data = pd.concat(data)
data.label.value_counts()

In [ ]:
data["tile_label"] = data.label.apply(lambda x: x.split(" (")[0])
data["tile_label"] = data["tile_label"].apply(lambda x: x if x != "NT" else "Normal")
data = data[data["tile_label"].isin(["TLS", "Normal"])]
data.tile_label.value_counts()

In [ ]:
sampleids = data[["file_id", "label"]].value_counts().reset_index().file_id.value_counts()
sampleids = sampleids[sampleids == 2]
sampleids.index[20:30]

In [ ]:
# Combine the groups into a list for easier iteration 
idx = np.arange(0, 9)
data = data.sample(frac=1)
sample_id = 58
sample_idx = (data.file_id == sampleids.index[sample_id]).values
group = data[sample_idx].query("tile_label == 'Normal'").iloc[idx]

In [ ]:
import matplotlib.pyplot as plt
import glob
import pyvips  # Assuming you use pyvips for image processing

# Set up the 3x3 grid
fig, axes = plt.subplots(3, 3, figsize=(10, 10))  # 3 rows, 3 columns

# Flatten the axes for easier indexing
axes = axes.flatten()

# Iterate through your group data and plot each image
for j, (_, cell) in enumerate(group.iterrows()):
    if j < len(axes):  # Prevent index errors if group has fewer than 9 items
        ax = axes[j]
        
        # Load the image
        img_path = glob.glob(f"../TCGA_KIRC/data/tcga_kirc_cryo/*/{cell.file_id}*")[0]
        image = pyvips.Image.new_from_file(img_path)
        
        # Crop the tile
        tile_array = crop_tile(image, cell.y, cell.x, 60 * 4)
        
        # Plot the tile
        ax.imshow(tile_array)
        #ax.set_title(str(cell.tile_label), fontsize=14)  # Adjust title size for readability
        ax.axis('off')  # Remove axis for cleaner visualization

# Remove unused subplots (if group has fewer than 9 items)
for ax in axes[len(group):]:
    ax.axis('off')

fig.suptitle(group.tile_label.values[0], fontsize=24, y=0.91)
# Adjust spacing between subplots
plt.subplots_adjust(wspace=0.05, hspace=0.05)  # Tweak spacing as needed

# Save and show the plot
plt.savefig("../plotting_notebooks/figures/Figure6E-kidney_normal_manual_anno.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Combine the groups into a list for easier iteration 
idx = np.arange(0, 9)
data = data.sample(frac=1)
sample_idx = (data.file_id == sampleids.index[sample_id]).values
group = data[sample_idx].query("tile_label == 'TLS'").iloc[idx]

In [ ]:
import matplotlib.pyplot as plt
import glob
import pyvips  # Assuming you use pyvips for image processing

# Set up the 3x3 grid
fig, axes = plt.subplots(3, 3, figsize=(10, 10))  # 3 rows, 3 columns

# Flatten the axes for easier indexing
axes = axes.flatten()

# Iterate through your group data and plot each image
for j, (_, cell) in enumerate(group.iterrows()):
    if j < len(axes):  # Prevent index errors if group has fewer than 9 items
        ax = axes[j]
        
        # Load the image
        img_path = glob.glob(f"../TCGA_KIRC/data/tcga_kirc_cryo/*/{cell.file_id}*")[0]
        image = pyvips.Image.new_from_file(img_path)
        
        # Crop the tile
        tile_array = crop_tile(image, cell.y, cell.x, 60 * 4)
        
        # Plot the tile
        ax.imshow(tile_array)
        #ax.set_title(str(cell.tile_label), fontsize=14)  # Adjust title size for readability
        ax.axis('off')  # Remove axis for cleaner visualization

# Remove unused subplots (if group has fewer than 9 items)
for ax in axes[len(group):]:
    ax.axis('off')
fig.suptitle(group.tile_label.values[0], fontsize=24, y=0.91)
# Adjust spacing between subplots
plt.subplots_adjust(wspace=0.05, hspace=0.05)  # Tweak spacing as needed

# Save and show the plot
plt.savefig("../plotting_notebooks/figures/Figure6E-kidney_tls_manual_anno.png", dpi=300, bbox_inches='tight')
plt.show()